# ALS runbook

## Loading libraries and setting parameters

In [1]:
import sys

sys.path.append("../../..")

from src.common import helpers, config
from src.models.als import ALS
from src.evaluation.evaluator import Evaluator

In [2]:
MODEL_NAME = "als"

paths = config.Paths(
    dataset_name="jobs_published-part",
    target_users_name="subset_validation",
    model_name=MODEL_NAME,
)

In [3]:
FACTORS = 400
REGULARIZATION = 0.1
ITERATIONS = 6
EVENT_WEIGHTS_MULTIPLIER = 100

N_RECOMMENDATIONS = 10

## Model training

In [4]:
model = ALS(
    factors=FACTORS,
    regularization=REGULARIZATION,
    iterations=ITERATIONS,
    event_weights_multiplier=EVENT_WEIGHTS_MULTIPLIER,
)

model.load_interactions(paths.train)

model.preprocess()

model.fit()

  0%|          | 0/6 [00:00<?, ?it/s]

## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS, show_progress=True
)

model.save_recommendations(recommendations, paths.recommendations)

17857it [00:09, 1920.75it/s]


## Evaluation

In [6]:
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.validation,
    k=10,
    models_to_evaluate=["als"],
)

evaluator.prepare()

evaluator.evaluate_models()

17857it [00:01, 16423.88it/s]


In [7]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
als,0.02375,0.130093,0.037678,0.027725,0.080122,0.053379,0.083473,0.564402,0.202442,0.820043,0.859354,8.391213,0.541879,0.0,0.0
